## Train ResNet34 model

In [1]:
from importlib.util import find_spec
if find_spec("model") is None:
    import sys
    sys.path.append('..')

In [2]:
from model.model import Resnet34
from data_loader.data_loaders import Cifar100DataLoader

In [3]:
dl = Cifar100DataLoader('../data', 32)

Files already downloaded and verified


In [4]:
data, target = next(iter(dl))
data.shape, target

(torch.Size([32, 3, 224, 224]),
 tensor([21, 87, 89, 98, 92, 98, 24, 82, 66, 49, 45, 82, 93, 14, 70, 69,  5, 38,
         21, 83, 90, 74,  8,  4, 65, 32, 22, 31,  4, 86, 75, 57]))

In [5]:
model = Resnet34()

In [6]:
model(data)

torch.Size([32, 3, 224, 224])
torch.Size([32, 64, 112, 112])
shape of x before g_avg_pooling: torch.Size([32, 512, 7, 7])
shape of x after g_avg_pooling: torch.Size([32, 512, 1, 1])
shape of x after squeeze: torch.Size([32, 512])


tensor([[ 0.0352, -0.2985,  0.1768,  ..., -0.1222,  0.1962, -0.3864],
        [-0.0241, -0.0800,  0.1294,  ..., -0.1697,  0.1536, -0.3195],
        [ 0.0155, -0.2548,  0.1520,  ..., -0.0833,  0.1382, -0.3929],
        ...,
        [ 0.0660, -0.4507,  0.1819,  ..., -0.0427,  0.1938, -0.4015],
        [ 0.0060, -0.2158,  0.1427,  ..., -0.1645,  0.2048, -0.3946],
        [ 0.0746, -0.3616,  0.1441,  ..., -0.0114,  0.1195, -0.2654]],
       grad_fn=<AddmmBackward>)